# Regime / Mixture 结构验证：rolling summary + clustering（仅用于验证）

这份 notebook 用来在时序数据上验证是否存在 **regime / mixture**（双峰/多峰背后的状态切换）。  
核心原则：**不对原始 X / y 直接聚类**，只在 **rolling summary 特征空间**聚类，并把聚类结果投影回时间轴与误差切片，用作结构性证据。

---

## 目标产出
- rolling mean/std 等 summary 特征
- KMeans / GMM 聚类标签（`cluster`）
- `cluster` 投影回时间轴的可视化
- `cluster` 切片下的 y 分布/残差分布对比
- （可选）当已有 baseline 预测时，比较各 cluster 下误差稳定性

---


In [ ]:
# === 基础依赖 ===
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## Step 0：数据约定（按需修改）

- `df`：主 DataFrame
- 时间索引：`df.index` 为递增的时间索引（DatetimeIndex 或整数时间步）
- 目标列：`TARGET_COL`
- （可选）已有 baseline 预测列：`PRED_COL`（用于残差分段评估）


In [ ]:
# === TODO: 载入数据 ===
# df = pd.read_csv("...", parse_dates=[...]).set_index("...")

# === TODO: 设置列名 ===
TARGET_COL = "y"          # 修改为真实目标列名
PRED_COL   = None         # 例如 "y_pred"；没有则保持 None

# sanity checks
assert TARGET_COL in df.columns, f"TARGET_COL={TARGET_COL} 不在 df.columns 中"
assert df.index.is_monotonic_increasing, "时间索引需要递增（单调不减）"

df[[TARGET_COL]].head()


## Step 1：构造 rolling summary 特征

rolling summary 特征用于近似描述“当前局部分布形态”（均值/波动/尾部等）。  
窗口大小按数据频率调整：例如分钟级可用 50/200，日频可用 20/60 等。


In [ ]:
# === rolling 特征窗口（按需修改）===
W1 = 50
W2 = 200

y = df[TARGET_COL].astype(float)

# rolling summary（只使用过去窗口；默认 rolling 使用当前点 + 过去 W-1 点，若需要严格不含当前点可 shift(1)）
df[f"{TARGET_COL}_mean_{W1}"] = y.rolling(W1).mean()
df[f"{TARGET_COL}_std_{W1}"]  = y.rolling(W1).std()

df[f"{TARGET_COL}_mean_{W2}"] = y.rolling(W2).mean()
df[f"{TARGET_COL}_std_{W2}"]  = y.rolling(W2).std()

# 可选：对尾部更敏感的 proxy（根据场景添加）
df[f"{TARGET_COL}_absmean_{W1}"] = y.abs().rolling(W1).mean()

# 特征列集合（保持低维：2~5 维更容易解释）
FEATURES = [
    f"{TARGET_COL}_mean_{W1}",
    f"{TARGET_COL}_std_{W1}",
    # f"{TARGET_COL}_mean_{W2}",
    # f"{TARGET_COL}_std_{W2}",
    f"{TARGET_COL}_absmean_{W1}",
]

# 取出用于聚类的特征表
Xr = df[FEATURES].dropna().copy()
Xr.head()


## Step 2：标准化（横截面）

在 summary 特征空间做聚类，先做标准化以避免某一维尺度主导距离。  
这一阶段仅用于结构验证，不参与最终预测模型训练，不涉及未来信息泄露的风险控制重点。


In [ ]:
scaler = StandardScaler()
Xs = scaler.fit_transform(Xr.values)
Xs.shape


## Step 3：聚类（KMeans / GMM）

- KMeans：快、稳定，适合作为第一版验证
- GMM：软分配、允许不同协方差形状，适合更接近 mixture 的假设

cluster 只用于验证，不把 `cluster` 直接当作预测模型特征。


In [ ]:
# === 选择聚类方法 ===
METHOD = "kmeans"   # "kmeans" 或 "gmm"
N_CLUSTERS = 2
RANDOM_STATE = 0

if METHOD == "kmeans":
    model = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE, n_init="auto")
    labels = model.fit_predict(Xs)
elif METHOD == "gmm":
    model = GaussianMixture(n_components=N_CLUSTERS, covariance_type="full", random_state=RANDOM_STATE)
    labels = model.fit_predict(Xs)
else:
    raise ValueError("METHOD must be 'kmeans' or 'gmm'")

Xr["cluster"] = labels
Xr["cluster"].value_counts().sort_index()


## Step 4：把 cluster 投影回原始时间轴（最直观的结构证据）

期望看到：cluster 在时间上成段出现（regime 切换），而不是高频随机交错。


In [ ]:
df_valid = df.loc[Xr.index].copy()
df_valid["cluster"] = Xr["cluster"]

plt.figure(figsize=(12, 4))
plt.scatter(df_valid.index, df_valid[TARGET_COL], c=df_valid["cluster"], s=6, alpha=0.8, cmap="tab10")
plt.title("Cluster labels projected back to time")
plt.xlabel("time")
plt.ylabel(TARGET_COL)
plt.show()


## Step 5：cluster 切片下的 y 分布对比

这里用 `describe()` 和分位数快速比较：均值、方差、尾部是否明显不同。


In [ ]:
summary = df_valid.groupby("cluster")[TARGET_COL].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).T
summary


## Step 6：误差/残差的 cluster 切片评估（可选）

当已有 baseline 预测 `PRED_COL` 时：
- 看不同 cluster 下残差均值（bias）与残差波动（variance）
- 若显著不同，说明全局模型在不同 regime 下存在系统性误差或尺度不匹配


In [ ]:
if PRED_COL is not None:
    assert PRED_COL in df_valid.columns, f"PRED_COL={PRED_COL} 不在 df.columns 中"
    df_valid["resid"] = df_valid[TARGET_COL] - df_valid[PRED_COL]

    resid_stats = df_valid.groupby("cluster")["resid"].describe(percentiles=[0.05, 0.5, 0.95]).T
    display(resid_stats)

    plt.figure(figsize=(12, 4))
    plt.scatter(df_valid.index, df_valid["resid"], c=df_valid["cluster"], s=6, alpha=0.8, cmap="tab10")
    plt.title("Residuals colored by cluster")
    plt.xlabel("time")
    plt.ylabel("resid")
    plt.show()
else:
    print("PRED_COL=None：跳过残差切片评估（需要已有 baseline 预测列）。")


## Step 7：用 cluster 验证，而不是直接用 cluster 训练（记录结论）

- 若 `cluster` 在时间上成段，且不同 cluster 下 y/残差统计显著不同：支持存在 regime / mixture
- 下一步的建模方向：
  1) 分段建模（按规则或按 proxy 阈值切分）
  2) 在单一模型里加入 rolling summary / 波动率 proxy 等“regime 特征”
  3) 对尺度漂移使用 rolling 标准化 / EWMA 标准化（严格只用过去窗口）

这里把结论写成几行 bullet，作为 presentation 的原始素材。


In [ ]:
# === 结论记录（手写）===
notes = []
notes.append("观察：cluster 在时间轴上是否呈现成段切换？（是/否）")
notes.append("统计：不同 cluster 下 y 的均值/方差/尾部是否显著不同？（是/否）")
notes.append("（可选）误差：不同 cluster 下 resid 的 std 是否显著不同？（是/否）")
notes.append("推断：是否支持 regime/mixture 假设？下一步采用（分段建模 / 加 regime 特征 / rolling 标准化）")

print("\n".join("- " + x for x in notes))


---

## 附：避免常见坑的检查清单（快速）

- rolling 特征 NaN 处理：`dropna()` 后 index 对齐
- 聚类特征维度尽量低（2~5维），减少噪声驱动的假分群
- 不在全量数据上生成“未来可见”的标签用于训练
- 如果 cluster 标签在时间上高度交错，直接放弃用 cluster 解释 regime（改用 rolling proxy 或不做 regime）
